In [1]:
# 1. SETUP
# Installs dependencies quietly
import sys
!{sys.executable} -m pip install -q -r memory_agent/requirements.txt
print("✅ Dependencies installed.")

# Add project to path
import os
sys.path.append(os.path.abspath("memory_agent"))
from core.agent import MemoryAgent
from langchain_community.embeddings import HuggingFaceEmbeddings
print("✅ Agent loaded successfully.")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 5.1.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.2 which is incompatible.
c:\Users\baish\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\langchain_core\_api\deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


✅ Dependencies installed.


c:\Users\baish\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Agent loaded successfully.


In [ ]:
# 2. SETUP & SIMULATION
import sys
import os
import json
import random
from datetime import datetime

# --- FIX: FORCE PYTHON TO LOOK IN THE CURRENT FOLDER ---
# This ensures we can find 'core' no matter how you launched Jupyter
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)
print(f"📂 Working Directory: {current_dir}")
# -------------------------------------------------------

# Now these imports will work
from core.agent import MemoryAgent
from core.retriever import Retriever 

# Initialize
print("🚀 Initializing Agent...")
agent = MemoryAgent()
# Explicitly set the retriever to use the Safe Hash version
agent.retriever = Retriever() 

# Define Needle
secret_code = "BlueMonkey99"
needle_text = f"My secret code is '{secret_code}'."

# Get Vector using the safe Hash method
print(f"🔹 Embedding Needle: '{needle_text}'")
real_vector = agent.retriever.embed_text(needle_text)

# Create Memories
memories = []
# Needle (Turn 1)
memories.append({
    "content": needle_text,
    "metadata": {"turn": 1, "category": "security", "timestamp": str(datetime.now())},
    "vector": real_vector
})

# Haystack (Distractors)
print("⏳ Generating 999 distractor memories...")
for i in range(2, 1001):
    text = f"User discussed topic #{i}."
    memories.append({
        "content": text,
        "metadata": {"turn": i, "category": "chitchat", "timestamp": str(datetime.now())},
        "vector": agent.retriever.embed_text(text) 
    })

# Save to Agent's memory
agent.memory_store.memories = memories
agent.memory_store._save_to_disk()
agent.turn_count = 1000

print(f"✅ SUCCESSFULLY INJECTED {len(memories)} MEMORIES.")
print("The system is now effectively at Turn #1,000.")

📂 Working Directory: c:\NeuroHack


ModuleNotFoundError: No module named 'current_dir'

In [2]:
# 3. ASK THE QUESTION
# We ask a question that requires finding the needle from Turn 1.
question = "What is my secret code?"

print(f"❓ User Question: '{question}'")
print("-" * 30)

# Run the Agent
response = agent.process_message(question)

print(f"🤖 AI Response: {response['response']}")

❓ User Question: 'What is my secret code?'
------------------------------
🤖 AI Response: Your secret code is 'BlueMonkey99'. I retrieved this information from our history, which was initially shared on Turn 1.


In [ ]:
# 4. UNDER THE HOOD
# Show exactly what the AI found in the database.
import pandas as pd

print("🔍 EVIDENCE: The Agent retrieved these specific memories:")
retrieved = response['active_memories']

df = pd.DataFrame([
    {
        "Turn": m['metadata']['turn'], 
        "Content": m['content'], 
        "Score": f"{m['score']:.4f}"
    } 
    for m in retrieved
])
display(df)

🔍 EVIDENCE: The Agent retrieved these specific memories:


,Turn,Content,Score
0,1,My secret code is 'BlueMonkey99'.,0.7265
1,127,User discussed topic #127.,0.2294
2,401,User discussed topic #401.,0.2149
